## Init

In [ ]:
%matplotlib inline
import sys
import os
import pandas as pd
import math
from time import sleep

In [ ]:
import logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO,
                    format="[%(name)s:%(levelname)-8s] -> %(message)s")
logger = logging.getLogger()

In [ ]:
pvpath = r"C:\Users\kruener\PycharmProjects\sysver\PyVerify"
if os.path.exists(pvpath):
    sys.path.insert(0, pvpath)

In [ ]:
from pverify.drivers.TemperatureChamber.thermonics_t2500 import thermonics_t2500

In [ ]:
thermo = thermonics_t2500(logger=logger)

In [ ]:
thermo.Initialize("GPIB0::5::INSTR", 1, 1, "")

In [ ]:
logger.info(thermo)

## Identity Feature

In [ ]:
logger.info("IDN:     " + thermo.Identity.Identifier)
logger.info("FW REV:  " + thermo.Identity.InstrumentFirmwareRevision)
logger.info("DRV REV: " + thermo.Identity.Revision)

## Temperature Logger

In [ ]:
logpath = r"C:\temp\templog.csv"
thermo.Templogger.config(logpath, 0.1)

In [ ]:
thermo.Templogger.start()
sleep(1)
thermo.Templogger.pause()
sleep(0.5)
thermo.Templogger.start()
sleep(1)
thermo.Templogger.stop()

In [ ]:
thermo.Templogger.as_dataframe()

In [ ]:
df = thermo.Templogger.as_dataframe()
df.plot(style=dict(AirTemp="bo-", DutTemp="ro-"))

## Properties

In [ ]:
# MODE

thermo.parameter.Mode = thermo.Enums.MODE.DUT_CTRL_K_TYPE
mode2 = thermo.parameter.Mode
thermo.parameter.Mode = thermo.Enums.MODE.DUT_CTRL_T_TYPE
mode3 = thermo.parameter.Mode
thermo.parameter.Mode = thermo.Enums.MODE.RTD
mode4 = thermo.parameter.Mode
thermo.parameter.Mode = thermo.Enums.MODE.AIR_CTRL
mode1 = thermo.parameter.Mode

logger.info(mode1)
logger.info(mode2)
logger.info(mode3)
logger.info(mode4)

assert mode1 == thermo.Enums.MODE.AIR_CTRL
assert mode2 == thermo.Enums.MODE.DUT_CTRL_K_TYPE
assert mode3 == thermo.Enums.MODE.DUT_CTRL_T_TYPE
assert mode4 == thermo.Enums.MODE.RTD

In [ ]:
# Temperature

logger.info("Dut Temperature: %.2f" % thermo.parameter.DutTemp)
logger.info("Air Temperature: %.2f" % thermo.parameter.AirTemp)

thermo.parameter.Mode = thermo.Enums.MODE.DUT_CTRL_K_TYPE
logger.info("Dut Temperature: %.2f" % thermo.parameter.Temp)
thermo.parameter.Mode = thermo.Enums.MODE.AIR_CTRL
logger.info("Air Temperature: %.2f" % thermo.parameter.Temp)

In [ ]:
# Setpoint

thermo.parameter.SetPoint = 23
rb = thermo.parameter.SetPoint
logger.info(rb)
assert rb == 23

In [ ]:
# Window

thermo.parameter.Window = 2
rb = thermo.parameter.Window
logger.info(rb)
assert rb == 2

In [ ]:
# RampRate

thermo.parameter.RampRate = 10
rb = thermo.parameter.RampRate
logger.info(rb)
assert rb == 10

thermo.parameter.RampRate = 999  # As fast as possible
rb = thermo.parameter.RampRate
logger.info(rb)
assert rb == 999

In [ ]:
# FlowRate

thermo.parameter.FlowRate = 5
rb = thermo.parameter.FlowRate
logger.info(rb)
assert math.ceil(rb) == 5

In [ ]:
# HeadPosition

thermo.parameter.HeadPosition = thermo.Enums.HEAD_POS.UP
rb = thermo.parameter.HeadPosition
logger.info(rb)
assert rb == thermo.Enums.HEAD_POS.UP

thermo.parameter.HeadPosition = thermo.Enums.HEAD_POS.DOWN
rb = thermo.parameter.HeadPosition
logger.info(rb)
assert rb == thermo.Enums.HEAD_POS.DOWN

In [ ]:
# FlowState

thermo.parameter.FlowState = True
rb = thermo.parameter.FlowState
logger.info(rb)
assert rb == True

thermo.parameter.FlowState = False
rb = thermo.parameter.FlowState
logger.info(rb)
assert rb == False

In [ ]:
# CompressorState

thermo.parameter.CompressorState = True
rb = thermo.parameter.CompressorState
logger.info(rb)
assert rb == True

thermo.parameter.CompressorState = False
rb = thermo.parameter.CompressorState
logger.info(rb)
assert rb == False

thermo.parameter.CompressorState = True
rb = thermo.parameter.CompressorState
logger.info(rb)
assert rb == True

## Wait until temperature settled

In [ ]:
logpath = r"C:\temp\templog.csv"
thermo.Templogger.config(logpath, 0.1)

In [ ]:
# Cannot be simulated
thermo.parameter.HeadPosition = thermo.Enums.HEAD_POS.DOWN
thermo.Templogger.start()
try:
    thermo.WaitForTemp(timeout_s=600, target=110., window_k=3, flowrate=6., soaktime_m=1, disable_exception=False)
except Exception as e:
    logger.exception(e)
finally:
    thermo.Templogger.stop()
    thermo.parameter.FlowState = False
    df = thermo.Templogger.as_dataframe()
    df.plot(style=dict(AirTemp="bo-", DutTemp="ro-"))

## Close

In [ ]:
thermo.Close()

# Dev area